# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?** 8.382332347441762
- **Question 2: What is the 13th number yielded** 3.605551275463989

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [9]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0

for sqrt_value in generator:
    print(sqrt_value)
    sum += sqrt_value

print(f'\nsum of first {limit} outputs is: ', sum)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979

sum of first 5 outputs is:  8.382332347441762


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.** 353.0




In [10]:
%%capture
!pip install dlt[duckdb]

In [12]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


import dlt

duckdb_conn = dlt.pipeline(destination='duckdb', dataset_name='people')

people_1_info = duckdb_conn.run(people_1(), table_name='people', write_disposition='replace')

print(people_1_info)

# people_2_info = duckdb_conn.run(people_2(), table_name='people', write_disposition='append')

print(people_2_info)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_colab_kernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_la

In [15]:
# show outcome

import duckdb

conn = duckdb.connect(f"{duckdb_conn.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{duckdb_conn.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people.people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

people_age = conn.sql("SELECT sum(age) FROM people").df()
display(people_age)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_2            │
└─────────────────────┘




 people.people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707280709.7064471,8sk9Um1ZwO8ufQ,None
1,2,Person_2,27,City_A,1707280709.7064471,SJofqvruKsTgSw,None
2,3,Person_3,28,City_A,1707280709.7064471,L6z2x3a6frEaww,None
3,4,Person_4,29,City_A,1707280709.7064471,twSGf+ca9vE7ig,None
4,5,Person_5,30,City_A,1707280709.7064471,Z+aWTq1+iU27cA,None
5,3,Person_3,33,City_B,1707280710.7185223,UB1IcA2R+DKBig,Job_3
6,4,Person_4,34,City_B,1707280710.7185223,sfhZNuizbHqfkg,Job_4
7,5,Person_5,35,City_B,1707280710.7185223,v6vT8h4gQy1zvQ,Job_5
8,6,Person_6,36,City_B,1707280710.7185223,KKd9JHx6K0HAGQ,Job_6
9,7,Person_7,37,City_B,1707280710.7185223,jE32h+6HSHioCQ,Job_7


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.** 213.0


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [16]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


import dlt

duckdb_conn = dlt.pipeline(destination='duckdb', dataset_name='people_merged')

people_1_info = duckdb_conn.run(people_1(), table_name='people_merged', write_disposition='replace')

print(people_1_info)

people_2_info = duckdb_conn.run(people_2(), table_name='people_merged', write_disposition='merge')

print(people_2_info)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_colab_kernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:////content/dlt_colab_ke

In [18]:
# show outcome

import duckdb

conn = duckdb.connect(f"{duckdb_conn.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{duckdb_conn.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people.people_merged table below:")

people = conn.sql("SELECT * FROM people_merged").df()
display(people)


people_age = conn.sql("SELECT sum(age) FROM people_merged").df()
display(people_age)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_merged       │
└─────────────────────┘




 people.people_merged table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,3,Person_3,33,City_B,1707281407.3711033,eM2FmBgalW6OmQ,Job_3
1,4,Person_4,34,City_B,1707281407.3711033,ZqS6r8p6CuSUeg,Job_4
2,5,Person_5,35,City_B,1707281407.3711033,EIgm2uHb0tchKg,Job_5
3,6,Person_6,36,City_B,1707281407.3711033,NaklQv7uTmsY2w,Job_6
4,7,Person_7,37,City_B,1707281407.3711033,F+psaY/1asGFxQ,Job_7
5,8,Person_8,38,City_B,1707281407.3711033,RzgwnoHQpiFisA,Job_8


,sum(age)
0,213.0


So merge inserts upsert data and also deletes new data when id does not match the new schema?

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX